In [75]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

In [5]:
data_path = '/disk/ru'
abnormal_user = pd.read_csv(os.path.join(data_path, 'abnormal_user'),header=None)
users = pd.read_json(os.path.join(data_path, 'users'), lines=True)

In [6]:
users.head()

,access,birth_year,country,gender,user_id
0,free,1989,8,female,093449f2282b4e4793948bbc3e9f1fd7278c0c49
1,premium,2001,4,female,3e9603e5ab69c5cfe14dcc1954c46cd849940b24
2,premium,1988,52,female,c12f6f2784787619d91f61cf215480b70d47f36e
3,premium,1995,16,female,d55e6dc8e7fdd87d42b639ed27ca95795738308b
4,premium,1996,44,female,6b1867e55d95cd39f0b3eac38518fd11572e732f


# preprocessing

In [8]:
users['abnormal'] = [0] * users.shape[0]

In [64]:
users['birth_year'].loc[users['birth_year']=='']=users['birth_year'].value_counts().index[0]
users['birth_year'] = users['birth_year'].astype('int')

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [69]:
y = users['abnormal']
X = users.drop(['abnormal','user_id'], axis=1)
for i in ['access', 'gender']:
    le = preprocessing.LabelEncoder()
    X[i] = le.fit_transform(X[i])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

## model 1 : basic features
use user profile as features

In [76]:
clf = RandomForestClassifier(n_jobs=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[529676,     13],
       [  6636,      2]])

## model2:  add track preference features
add user preference of track to classify abnormal user

In [89]:
user_item =  pd.read_csv(os.path.join(data_path, 'user_item'))

In [84]:
users = users.sort_values(by='user_id')

In [90]:
users.reset_index(drop=True,inplace=True)
user_item.reset_index(drop=True,inplace=True)

In [92]:
user_item = pd.concat([users, user_item], axis=1, ignore_index=True)

In [96]:
user_item.rename(columns={0: 'access',1:'birth_year',2:'country',3:'gender',4:'user_id',5:'abnormal',56:'user_id2'}, inplace=True)

In [98]:
y = user_item['abnormal']
X = user_item.drop(['user_id','user_id2','abnormal'],axis=1)
for i in ['access', 'gender']:
    le = preprocessing.LabelEncoder()
    X[i] = le.fit_transform(X[i])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [99]:
clf = RandomForestClassifier(n_jobs=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[529124,    491],
       [  6360,    352]])

### conclusion: add track preference get better result